In [1]:
import os
import torch

In [2]:
os.getcwd()

'/Users/tzikos/Desktop/DTU/MLOps/Project/Plant_Leaves_Classification_MLOps_DTU02476/notebooks'

In [15]:
pt = torch.load('../data/processed/test/targets.pt')

/var/folders/4f/7kcbgj992s5fhk7wcryyv4hc0000gn/T/ipykernel_11656/1032847024.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pt = torch.load('../data/processed/test/targe

In [16]:
pt.shape

torch.Size([15])